<a href="https://colab.research.google.com/github/depaimonn/Diagnosa-Penyakit-Tanaman-Padi/blob/main/Diagnosa_Penyakit_Tanaman_Padi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import textwrap

In [ ]:
gejala = {
    'G1': 'Daun menguning', 'G2': 'Bercak coklat pada daun', 'G3': 'Daun mengering',
    'G4': 'Batang busuk dan berbau', 'G5': 'Pertumbuhan terhambat', 'G6': 'Daun berlubang',
    'G7': 'Daun berkerut', 'G8': 'Tanaman kerdil'
}
penyakit = {
    'P1': 'Hawar Daun Bakteri', 'P2': 'Blas', 'P3': 'Busuk Batang', 'P4': 'Tungro'
}

In [ ]:
rules = [
    ('G1', 'P1', 0.6), ('G2', 'P1', 0.8), ('G6', 'P1', 0.7), # Hawar Daun Bakteri
    ('G2', 'P2', 0.7), ('G3', 'P2', 0.8), ('G5', 'P2', 0.6), # Blas
    ('G4', 'P3', 0.9), ('G5', 'P3', 0.7), ('G1', 'P3', 0.5), # Busuk Batang
    ('G1', 'P4', 0.8), ('G7', 'P4', 0.9), ('G8', 'P4', 0.85) # Tungro
]

rekomendasi = {
    'P1': 'Gunakan bakterisida, lakukan sanitasi lingkungan, dan gunakan varietas padi yang tahan.',
    'P2': 'Gunakan fungisida, atur jarak tanam, dan hindari pemupukan Nitrogen (N) berlebihan.',
    'P3': 'Lakukan pengeringan lahan secara berkala, gunakan fungisida yang sesuai, dan bakar jerami sisa panen.',
    'P4': 'Kendalikan vektor wereng hijau dengan insektisida dan lakukan eradikasi tanaman yang terinfeksi.'
}

In [ ]:
# Perhitungan Certainty Factor dan Forward Chaining (Sesuai Rubrik 2 & 3)
def forward_chaining(user_cf):
    """
    Mengevaluasi semua rules (forward chaining) untuk menghitung CF penyakit
    berdasarkan CF gejala dari user.
    """
    cf_penyakit = {p_code: 0.0 for p_code in penyakit.keys()}

    for gej_code, pen_code, cf_rule in rules:
        cf_user = user_cf.get(gej_code, 0.0)
        if cf_user > 0:
            # Formula CF(H,E) = CF(user) × CF(rule)
            cf_evidence = cf_user * cf_rule

            # Formula kombinasi: CF_kombinasi = CF_old + CF_new × (1 - CF_old)
            cf_old = cf_penyakit[pen_code]
            cf_penyakit[pen_code] = cf_old + cf_evidence * (1 - cf_old)

    return cf_penyakit

In [ ]:
# User Interface (Sesuai Rubrik 4)
def get_user_input():
    """Meminta input CF dari user dengan validasi."""
    user_cf = {}
    print("="*60)
    print("Masukkan Tingkat Keyakinan Anda untuk Setiap Gejala (0.0 - 1.0)")
    print("0.0 = Tidak Ada | 0.4 = Mungkin | 0.8 = Hampir Pasti | 1.0 = Pasti")
    print("="*60)
    for code, desc in gejala.items():
        while True:
            try:
                val = float(input(f"-> {desc} ({code})? [0.0-1.0]: "))
                # Validasi input CF user (0-1)
                if 0.0 <= val <= 1.0:
                    user_cf[code] = val
                    break
                else:
                    print("ERROR: Harap masukkan angka antara 0.0 dan 1.0.")
            except ValueError:
                print("ERROR: Input tidak valid. Harap masukkan angka.")
    return user_cf

def run_diagnose():
    """Fungsi utama untuk menjalankan alur diagnosa interaktif."""
    user_cf = get_user_input()
    cf_results = forward_chaining(user_cf)

    # Filter penyakit dengan CF > 0 untuk ditampilkan
    diagnosed_diseases = {p: cf for p, cf in cf_results.items() if cf > 0}

    if not diagnosed_diseases:
        print("\n--- HASIL DIAGNOSA ---\nTidak ada penyakit yang terdeteksi berdasarkan gejala yang diberikan.")
        return

    # Ranking penyakit berdasarkan CF tertinggi
    ranked_diseases = sorted(diagnosed_diseases.items(), key=lambda item: item[1], reverse=True)

    print("\n" + "="*25 + " HASIL DIAGNOSA " + "="*25)
    print("Ranking Penyakit Berdasarkan Tingkat Keyakinan:")
    for i, (p_code, cf) in enumerate(ranked_diseases):
        print(f"{i+1}. {penyakit[p_code]:<25} | CF: {cf:.2%} ({cf:.4f})")

    # Kesimpulan diagnosa dan rekomendasi
    top_disease_code = ranked_diseases[0][0]
    top_disease_name = penyakit[top_disease_code]
    top_disease_cf = ranked_diseases[0][1]

    print("\n--- KESIMPULAN ---")
    print(f"Berdasarkan analisis, penyakit yang paling mungkin adalah:")
    print(f"** {top_disease_name} ** dengan tingkat keyakinan {top_disease_cf:.2%}.")

    print("\n--- REKOMENDASI ---")
    print(textwrap.fill(rekomendasi[top_disease_code], 70))
    print("="*67)

In [ ]:
# Testing & Code Quality (Sesuai Rubrik 5)
def run_test_cases():
    """Menjalankan 3 skenario uji coba yang berbeda secara otomatis."""
    test_cases = [
        {"desc": "Kasus 1: Gejala kuat mengarah ke Tungro (P4)",
         "input": {'G1': 0.8, 'G7': 1.0, 'G8': 0.9, 'G2': 0, 'G3': 0, 'G4': 0, 'G5': 0.2, 'G6': 0}},
        {"desc": "Kasus 2: Gejala tumpang tindih antara Hawar Daun (P1) dan Blas (P2)",
         "input": {'G1': 0.6, 'G2': 0.9, 'G3': 0.7, 'G5': 0.5, 'G6': 0.8, 'G4': 0, 'G7': 0, 'G8': 0}},
        {"desc": "Kasus 3: Gejala kuat mengarah ke Busuk Batang (P3)",
         "input": {'G4': 1.0, 'G5': 0.7, 'G1': 0.5, 'G2': 0, 'G3': 0, 'G6': 0, 'G7': 0, 'G8': 0}}
    ]

    for i, case in enumerate(test_cases):
        print(f"\n--- MENJALANKAN TEST CASE {i+1}: {case['desc']} ---")
        results = forward_chaining(case['input'])
        ranked = sorted(results.items(), key=lambda item: item[1], reverse=True)
        print("Hasil Kalkulasi CF:")
        for p_code, cf in ranked:
            if cf > 0:
                print(f"- {penyakit[p_code]:<25} | CF: {cf:.4f}")
        print("-" * 55)

# Program Utama
if __name__ == "__main__":
    while True:
        print("\n=== SISTEM PAKAR DIAGNOSA PENYAKIT PADI (CF) ===")
        print("1. Mulai Diagnosa Interaktif")
        print("2. Jalankan Skenario Uji Coba Otomatis")
        print("3. Keluar")
        choice = input("Pilih menu [1/2/3]: ")
        if choice == '1':
            run_diagnose()
        elif choice == '2':
            run_test_cases()
        elif choice == '3':
            print("Program selesai.")
            break
        else:
            print("Pilihan tidak valid.")


=== SISTEM PAKAR DIAGNOSA PENYAKIT PADI (CF) ===
1. Mulai Diagnosa Interaktif
2. Jalankan Skenario Uji Coba Otomatis
3. Keluar
Pilih menu [1/2/3]: 2

--- MENJALANKAN TEST CASE 1: Kasus 1: Gejala kuat mengarah ke Tungro (P4) ---
Hasil Kalkulasi CF:
- Tungro                    | CF: 0.9915
- Busuk Batang              | CF: 0.4840
- Hawar Daun Bakteri        | CF: 0.4800
- Blas                      | CF: 0.1200
-------------------------------------------------------

--- MENJALANKAN TEST CASE 2: Kasus 2: Gejala tumpang tindih antara Hawar Daun (P1) dan Blas (P2) ---
Hasil Kalkulasi CF:
- Hawar Daun Bakteri        | CF: 0.9212
- Blas                      | CF: 0.8860
- Busuk Batang              | CF: 0.5450
- Tungro                    | CF: 0.4800
-------------------------------------------------------

--- MENJALANKAN TEST CASE 3: Kasus 3: Gejala kuat mengarah ke Busuk Batang (P3) ---
Hasil Kalkulasi CF:
- Busuk Batang              | CF: 0.9617
- Blas                      | CF: 0.4200
- 